In [19]:
from pymorphy2 import MorphAnalyzer
import stanza
stanza.download('ru', processors='tokenize,pos')
nlp = stanza.Pipeline('ru', processors='tokenize,pos')
m = MorphAnalyzer()

2021-10-02 23:24:20 INFO: Downloading these customized packages for language: ru (Russian)...
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| pos       | syntagrus |
| pretrain  | syntagrus |

2021-10-02 23:24:20 INFO: File exists: C:\Users\Acer\stanza_resources\ru\tokenize\syntagrus.pt.
2021-10-02 23:24:20 INFO: File exists: C:\Users\Acer\stanza_resources\ru\pos\syntagrus.pt.


2021-10-02 23:24:21 INFO: File exists: C:\Users\Acer\stanza_resources\ru\pretrain\syntagrus.pt.
2021-10-02 23:24:21 INFO: Finished downloading models and saved to C:\Users\Acer\stanza_resources.
2021-10-02 23:24:21 INFO: Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| pos       | syntagrus |

2021-10-02 23:24:21 INFO: Use device: cpu
2021-10-02 23:24:21 INFO: Loading: tokenize
2021-10-02 23:24:21 INFO: Loading: pos
2021-10-02 23:24:22 INFO: Done loading processors!


Сначала прогоняем текст через Pymorphy и Stanza:

In [201]:
import nltk
with open ('text.txt', encoding = 'utf-8') as f:
    text = f.read()
    py_tuples = []
    pr_word = ''
    tokens = nltk.word_tokenize(text)
    for token in tokens:
        if token not in '.-?,!:;—»«':
            p = m.parse(token)
            pos = p[0].tag.POS
            if p[0].word != pr_word:
                t = (p[0].word, pos)
                py_tuples.append(t)
                pr_word = p[0].word
    st_tuples = []
    doc = nlp(text)
    for sentence in doc.sentences:
        for word in sentence.words:
            t = (word.text, word.pos)
            st_tuples.append(t)

Затем через Mystem:

In [3]:
with open ('text.txt', encoding = 'utf-8') as f:
    text = f.read()
    from pymystem3 import Mystem
    m = Mystem()
    ana = m.analyze(text)
stem_tuples = []
for word in ana:
    if 'analysis' in word:
        if word['analysis']:
            gr = word['analysis'][0]['gr']
            pos = gr.split('=')[0].split(',')[0]
            stem_tuples.append((word['text'], pos))
        else:
            stem_tuples.append((word['text'], ''))

NameError: name 'stem_tags' is not defined

Создаем списки с тегами:

In [203]:
import csv
right_tags = []
with open('corpus.csv', encoding = 'utf-8') as File:
    reader = csv.DictReader(File, delimiter = ',')
    for row in reader:
        right_tags.append(row['tag'])
stanza_tags = []
for word in st_tuples:
    if word[1] != 'PUNCT':
        stanza_tags.append(word[1])
py_tags = []
for word in py_tuples:
    py_tags.append(word[1])
stem_tags = []
for word in stem_tuples:
    if word[0] == 'А':
        stem_tags.append('N')
    if word[0] == 'бе' or word[0] == 'ве' or word[0] == 'ге' or word[0] == 'де' or word[0] == 'е' or word[0] == 'же' or word[0] == 'зе' or word[0] == 'кома':
        continue
    stem_tags.append(word[1])

Приводим все к единому тагсету и считаем accuracy:

In [208]:
from sklearn.metrics import accuracy_score
new_stem_tags = []
nnew_stanza_tags = []
for tag in stem_tags:
    if tag == 'S':
        new_stem_tags.append('NOUN')
    elif tag == 'V':
        new_stem_tags.append('VERB')
    elif tag == 'A':
        new_stem_tags.append('ADJ')
    elif tag == 'PR':
        new_stem_tags.append('PREP')
    elif tag == 'APRO':
        new_stem_tags.append('ADJF')
    elif tag == 'SPRO':
        new_stem_tags.append('NPRO')
    else:
        new_stem_tags.append(tag)
for tag in stanza_tags:
    if tag == 'ADP':
        nnew_stanza_tags.append('PREP')
    elif tag == 'CCONJ':
        nnew_stanza_tags.append('CONJ')
    elif tag == 'DET':
        nnew_stanza_tags.append('ADJF')
    else:
        nnew_stanza_tags.append(tag)
new_py_tags = []
for tag in py_tags:
    if tag == 'ADJF' or tag == 'ADJS':
        new_py_tags.append('ADJ')
    elif tag == None:
        new_py_tags.append('')
    elif tag == 'PRCL':
        new_py_tags.append('PART')
    elif tag == 'ADVB':
        new_py_tags.append('ADV')
    elif tag == 'GRND' or tag == 'PRTF' or tag == 'INFN':
        new_py_tags.append('ADV')
    else:
        new_py_tags.append(tag)
new_right_tags = []
for tag in right_tags:
    if tag == 'ADJF' or tag == 'ADJS':
        new_right_tags.append('ADJ')
    elif tag == 'PRCL':
        new_right_tags.append('PART')
    elif tag == 'ADVB':
        new_right_tags.append('ADV')
    elif tag == 'GRND' or tag == 'PRTF' or tag == 'INFN':
        new_right_tags.append('ADV')
    else:
        new_right_tags.append(tag)
stem_accuracy = accuracy_score(new_stem_tags, new_right_tags)
py_accuracy = accuracy_score(new_py_tags, new_right_tags)
stanza_accuracy = accuracy_score(nnew_stanza_tags, new_right_tags)

0.7929515418502202
0.8766519823788547


Ищем н-граммы с помощью Pymorphy:

In [214]:
prep_adj_nouns = []
prep_adj_noun = []
adv_verbs = []
adv_verb = []
verb_noun = []
verb_nouns = []
for ind in range(len(py_tags)):
    if ind == len(py_tags)-1:
        pass
    elif py_tuples[ind][1] == 'PREP' and py_tuples[ind+1][1] == 'ADJF' and py_tuples[ind+2][1] == 'NOUN':
        prep_adj_noun.append(py_tuples[ind][0])
        prep_adj_noun.append(py_tuples[ind+1][0])
        prep_adj_noun.append(py_tuples[ind+2][0])
        prep_adj_nouns.append(prep_adj_noun)
        prep_adj_noun = []
    if ind == len(py_tags)-1:
        pass
    elif py_tuples[ind][1] == 'ADVB' and py_tuples[ind+1][1] == 'VERB':
        adv_verb.append(py_tuples[ind][0])
        adv_verb.append(py_tuples[ind+1][0])
        adv_verbs.append(adv_verb)
        adv_verb = []
    elif py_tuples[ind][0] == 'не' and py_tuples[ind+1][1] == 'VERB':
        verb_noun.append(py_tuples[ind][0])
        verb_noun.append(py_tuples[ind+1][0])
        verb_nouns.append(verb_noun)
        verb_noun = []

[['на', 'верхний', 'этаж'], ['от', 'страшной', 'картины'], ['в', 'крученыховском', 'аде']]


В предыдущем задании можно использовать н-грамму не+прилаг/наречие (для негативных отзывов) и н-грамму очень+прилаг (для положительных), что я и попыталась реализовать в предыдущем коде